:::{.callout-important}
## 시그모이드 함수 사용 이유

단위 계단 함수(unit step function)은 불연속이므로 미분할 수 없는 지점이 있어 퍼셉트론 학습에 사용하기에 적합하지 않다. 0과 1 사의의 연속적인 값을 가지는 시그모이드(sigmoid) 함수를 사용한다. p214
:::

:::{.callout-important}
다층 퍼셉트론(multi-layer perceptron)에서는 입력 데이터도 하나의 층으로 간주한다. p217
:::

:::{.callout-important}
## 오차 역전파

오차 역전파(back-propagation)는 출력층에서 입력층 방향으로 뒤에서부터 오차를 추적하며 경사 하강법을 이용해 가중치와 편향을 수정하는 수학적 방법이다. 오차 역전파의 수학적 원리는 연쇄 법칙(chain rule)과 편미분(partial derivative)이다. 신경망은 구조가 복잡해서 손실 함수의 기울기를 계산하고 경사 하강법으로 가중치와 편향을 수정하는 과정의 연산량이 많기 때문에 이 연산 과정을 수학적으로 개선한 방법이 오차 역전파다. p218
::: 

:::{.callout-important}
## 기울기 소실(gradient vanishing) 문제

오차 역전파의 계산 과정에서 층마다 활성화 함수의 기울기를 곱하는데, 활성화 함수로 사용하는 시그모이드 함수의 기울기는 0.3 미만이다. 따라서 은닉층을 지날 때마다 0.3 미만의 값을 반복해서 곱하다 보면 입력층에 가까워졌을 때는 기울기가 0에 가까워진다. 가중치를 수정할 때는 기울기의 크기에 학습률을 곱한 만큼 가중치를 수정하게 되는데, 기울기가 0에 가까우면 기울기의 크기에 학습률은 곱한 결과도 0에 가까우므로 가중치가 거의 수정되지 않아 제대로 학습이 되지 않는다. 이를 해결하기 위해 은닉층 활성화 함수로 시그모이드 함수 대신 렐루 함수를 사용한다. 렐루 함수는 값이 0을 넘을 때는 미분 값이 항상 1이기 때문에 많은 은닉층이 있어도 기울기가 소실되지 않고 잘 전달된다.
:::

## 활성화 함수

| 사용층 | 용도 | 활성화 함수 | 설명 |
|:-:|:-:|:-:|-|
|은닉층| 기울기 소실 문제를 줄이고 다음 층으로 신호 전달 | 렐루 / 리키 렐루 | 0이하는 모두 0, 0 초과는 항상 1 |
|출력층| 이진 분류 | 시그모이드 | 양성일 확률로 따져 양성, 음성 판정 |
| | 다중 분류 | 소프트맥스 | 각 클래스에 속할 확률을 따져 분류 |
| | 회귀 | - | 노드 값을 그대로 출력 |

## 손실 함수

| 용도 | 손실 함수 |
|:-:|-|
| 이진 분류 | 이진 크로스 엔트로피 |
| 다중 분류 | 크로스 엔트로피 |
| 회귀 | 평균 제곱 오차|

## 옵티마이저(optimizer) - 사용하는 경사 하강법 종류

| 옵티마이저 | 특징 |
|-|-|
| 확률적 경사 하강법(SGD, Stochastic Gradient Descent) | 데이터 샘플을 무작위로 추출하여 일부만 경사 하강법을 사용해 학습 속도 개선|
| RMSProp(Root Mean Square Propagation) | 학습률이 커서 최적 가중치를 지나치는 문제 해결<br>손실 함수의 기울기가 크면 학습률이 크고 작으면 학습률이 작아지도록 조절 |
| 모멘텀(momentum) | 지역 최솟값에 갇히는 문제를 해결하기 위해<br>손실 함수의 기울기가 0인 여러 지점 중 한 지점에서 멈추지 않고<br>계속 최솟값을 찾도록 관성 개념을 추가 |
| 아담(adam) | RMSProp과 모멘텀 기능을 합침 |

:::{.callout-important}
## 텐서(tensor) 자료구조

넘파이 배열과 구조적, 기능적으로 거의 비슷하지만 그래픽 카드를 사용해 병렬 연산을 할 수 있다는 차이가 있다. NVIDIA 그래픽 카드가 제공하는 쿠다(cuda) 환경에서 모델을 학습할 수 있다. 
:::

## 선형 회귀 구현


In [1]:
from sklearn import datasets
import torch
import torch.nn as nn
import torch.optim as optim # 옵티마이저

In [3]:
dataset = datasets.load_boston()

X, y = dataset['data'], dataset['target']

### 특성과 타깃을 텐서 자료구조로 변환

- 실수형 텐서: torch.FloatTensor
- 정수형 텐서: torch.LongTensor
- 파이토치는 타겟을 2차원 배열로 인식
- 차원 추가 메서드 : unsqueeze() eg. 맨 뒤에 길이가 1인 차원 추가 unsqueeze(-1)
- 차원 제거 메서드 : squeeze

In [4]:
X = torch.FloatTensor(X)
y = torch.FloatTensor(y).unsqueeze(-1)

In [5]:
X.shape, y.shape

(torch.Size([506, 13]), torch.Size([506, 1]))

### 표준화

sklearn의 StandardSacler는 2차원 배열까지만 지원하기 때문에 3차원 이상의 데이터를 많이 다루는 딥러닝에서는 표준화를 직접 하는 것에 익숙해져야 한다.

In [6]:
X = (X - torch.mean(X))/torch.std(X)

### 선형 회귀 모델 객체 생성

(입력값, 출력값) = (특성 13개, 타겟 1개)

- 활성화 함수 => 시그모이드 함수 => 로지스틱 회귀
- 활성화 함수 없는 퍼셉트론 => 선형 회귀

In [7]:
model = nn.Linear(13, 1)

### 손실함수와 옵티마이저 객체 생성

- MSE 손실 함수 -> criterion 변수 이름이 관례
- 확률적 경사 하강법 옵티마이저 
- parameters 메서드로 모델의 가중치를 불러올 수 있다.

In [8]:
criterion = nn.MSELoss()

optimizer = optim.SGD(model.parameters(), lr=0.01)

### 학습 함수 정의(에포크 1회) -> 오차(손실) 반환

1. 손실 함수로 오차 계산(실제 타깃과 모델 예측 타깃의 차이)
1. 미분으로 손실 함수 기울기 계산하여 가중치를 어떻게 수정해야 오차가 줄어드는지 파악
1. 학습률만큼 가중치 수정

한 번의 에포크는 주어진 모든 데이터 샘플을 이용해 한 번 학습하는 것을 의미

In [9]:
def train(model, criterion, optimizer, X, y):
    
    optimizer.zero_grad()           # 기울기 초기화
    hypothesis = model(X)           # 모델 사용해 타깃 예측
    loss = criterion(hypothesis, y) # 오차 계산
    loss.backward()                 # 기울기 계산
    optimizer.step()                # 경사하강법으로 가중치 수정
    return loss.item()              # 현재 에포크 오차 반환 -> 파이썬 숫자형 자료구조로 변환(item)
    

### 학습하며 오차 출력

In [10]:
n_epochs = 100 # 학습 횟수
for epoch in range(1, n_epochs):
    loss = train(model, criterion, optimizer, X, y)
    if epoch % 10 == 0:
        print('epoch: {}, loss: {:.4f}'.format(epoch, loss))

epoch: 10, loss: 115.4186
epoch: 20, loss: 97.9571
epoch: 30, loss: 88.5269
epoch: 40, loss: 82.5146
epoch: 50, loss: 78.6239
epoch: 60, loss: 76.0548
epoch: 70, loss: 74.3105
epoch: 80, loss: 73.0827
epoch: 90, loss: 72.1799


## 로지스틱 회귀

- 이진 크로스 엔트로피 손실 함수 사용
- view 메서드는 넘파이의 reshape와 유사

### 데이터 불러오기, 텐서 변환, 표준화

In [14]:
from sklearn import datasets
import torch
import torch.nn as nn
import torch.optim as optim

dataset = datasets.load_breast_cancer()

X, y = dataset['data'], dataset['target']

X = torch.FloatTensor(X)
y = torch.FloatTensor(y).view(-1, 1)

X = (X - torch.mean(X)) / torch.std(X)

X.shape, y.shape

(torch.Size([569, 30]), torch.Size([569, 1]))

### nn.Sequential

Sequential 클래스 생성자에 모델 객체와 활성화 함수 객체를 전달하면 두 객체를 차례대로 사용해 계산한 결과를 반환하는 모델 생성

In [15]:
model = nn.Sequential(
    nn.Linear(30, 1),
    nn.Sigmoid()
)

### 손실함수, 옵티마이저 정의 / 학습 함수 정의 / 학습

이진 크로스 엔트로피(Binary Cross Entropy) 손실함수

In [16]:
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

def train(model, criterion, optimizer, X, y):
    optimizer.zero_grad()
    hypothesis = model(X)
    loss = criterion(hypothesis, y)
    loss.backward()
    optimizer.step()
    return loss.item()

n_epochs = 100
for epoch in range(1, n_epochs + 1):
    loss = train(model, criterion, optimizer, X, y)
    if epoch % 10 == 0:
        print('epoch: {}, loss: {:.4f}'.format(epoch, loss))

epoch: 10, loss: 0.6046
epoch: 20, loss: 0.5323
epoch: 30, loss: 0.4806
epoch: 40, loss: 0.4422
epoch: 50, loss: 0.4126
epoch: 60, loss: 0.3891
epoch: 70, loss: 0.3701
epoch: 80, loss: 0.3543
epoch: 90, loss: 0.3411
epoch: 100, loss: 0.3298


### 타깃 예측 및 평가

In [17]:
y_predicted = (model(X) >= 0.5).float()

score = (y_predicted == y).float().sum()
print('accuracy: {:.2f}'.format(score))

accuracy: 518.00


## 클래스로 모델 정의

- 파이토치에서 모델 클래스를 정의할 때는 nn.Module 클래스를 상속받아야 한다.
- 모델 구조는 `__init__` 생성자 안에 정의한다.

### 선형회귀 모델 클래스 정의

In [18]:
class LinearRegression(nn.Module):
    def __init__(self, num_features): # 생성자에서 모델 구조 정의
        super().__init__()
        self.linear = nn.Linear(num_features, 1)
    
    def forward(self, X): # 순전파 정의
        out = self.linear(X)  # 생성자에서 만든 선형모델로 타깃 예측 및 반환
        return out
    
model = LinearRegression(13)

### 로지스틱 회귀 모델 클래스 정의

In [20]:
class LogisticRegression(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.linear = nn.Linear(num_features, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, X):
        out = self.linear(X)
        out = self.sigmoid(out)
        return out
    
model = LogisticRegression(30)

## 배치(batch) 학습

전체 데이터에 대한 추론과 오차 역전파를 한 번에 실행하기에는 컴퓨터 메모리 한계가 존재한다. 이 문제를 해결하기 위해 데이터를 나눠서 처리하는 배치라는 개념이 등장한다. 배치란 한 번의 에포크를 수행할 때 처리하는 샘플의 개수를 의미한다. 전체 데이터 샘플을 지정된 배치 크기로 나눠서 학습하고 마지막 배치 학습이 끝나면 한 번의 에포크가 수행된 것이다.

배치 학습을 위해 DataLoader 클래스 제공

### 데이터 로드, 표준화

In [24]:
from sklearn import datasets
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

dataset = datasets.load_breast_cancer()
X, y = dataset['data'], dataset['target']
X = torch.FloatTensor(X)
y = torch.FloatTensor(y).view(-1,1)

X = (X - torch.mean(X)) / torch.std(X)

### 배치 학습 구현 위해 데이터와 타깃 합치기



In [25]:
# 입력 데이터와 타깃을 묶어 텐서 데이터셋 생성
dset = TensorDataset(X, y)

# 한 번에 256개의 데이터 샘플을 배치로 사용하는 데이터로더 생성
loader = DataLoader(dset, batch_size=256, shuffle=True)

### 신경망 모델 클래스 정의 / 학습 함수 정의 / 학습 / 평가

In [29]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.linear1 = nn.Linear(num_features, 4) # 은닉층 노드 4개 생성
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(4, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, X):
        out = self.linear1(X)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.sigmoid(out)
        return out

model = NeuralNetwork(30) 
criterion = nn.BCELoss()  # 이진 크로스 엔트로피
optimizer = optim.SGD(model.parameters(), lr=0.1)

# 학습 함수 정의
def train(model, criterion, optimizer, loader):
    epoch_loss = 0    # 현재 에포크의 오차 저장 변수
    
    for X_batch, y_batch in loader:
        optimizer.zero_grad()
        hypothesis = model(X_batch)
        loss = criterion(hypothesis, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() # 현재 배치의 오차 누적
        
    return epoch_loss / len(loader) # 현대 에포크의 오차 평균 반환

# 모델 학습
n_epochs = 100
for epoch in range(1, n_epochs + 1):
    loss = train(model, criterion, optimizer, loader)
    if epoch % 10 == 0:
        print('epoch: {}, loss: {:.4f}'.format(epoch, loss))
        
# 학습된 모델로 타깃 예측
y_predicted = (model(X) >= 0.5).float()

# 정확도 계산
score_trained_model = (y_predicted == y).float().mean()
print('accuracy: {:.2f}'.format(score_trained_model))

epoch: 10, loss: 0.5338
epoch: 20, loss: 0.3442
epoch: 30, loss: 0.2628
epoch: 40, loss: 0.2579
epoch: 50, loss: 0.2114
epoch: 60, loss: 0.1996
epoch: 70, loss: 0.2114
epoch: 80, loss: 0.2221
epoch: 90, loss: 0.2065
epoch: 100, loss: 0.2161
accuracy: 0.92


### 모델 저장

- model.state_dict() 모델 정보를 가진 딕셔너리에 접근
- torch.save() 모델 저장 .pt 또는 .pth

In [30]:
torch.save(model.state_dict(), 'breast_cancer.pt')

### 모델 로드, 예측

- torch.load() 모델 불러오기

In [31]:
# 신경망 모델의 객체 생성
loaded_model = NeuralNetwork(30)

# 저장한 파일에서 가중치 불러와 복원
loaded_model.load_state_dict(torch.load('breast_cancer.pt'))

# 타깃 예측
y_pred2 = (loaded_model(X) >= 0.5).float()

score_loaded_model = (y_pred2 == y).float().mean()
print('accuracy of loaded model: {:.2f}'.format(score_loaded_model))

accuracy of loaded model: 0.92
